In [ ]:
!pip install langchain langchain-openai

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/온라인해커톤

/content/drive/MyDrive/Colab Notebooks/온라인해커톤


In [ ]:
import os
with open('./key/openai_api_key', 'r') as f:
    api_key = f.read().strip()

os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Literal

def build_time_options(step=30):
    return [f"{h:02d}:{m:02d}" for h in range(24) for m in range(0, 60, step)]

TIME_OPTIONS = set(build_time_options(30))

class UserProfile(BaseModel):
    # 온보딩 정보
    mobility_issue: bool = Field(default=True, description="거동 불편 여부")
    living_arrangement: Literal['alone', 'with_family'] = Field(default='alone', description="가족 동거 여부")
    wake_up_time: str = Field(default="07:00", description="기상 시간") #문자열로 받고, 필요할 때 변환
    bed_time: str = Field(default="21:00", description="취침 시간")

    @validator('wake_up_time', 'bed_time')
    def validate_time_format(cls, v):
        if v not in TIME_OPTIONS:
            raise ValueError(f"시간은 30분 단위여야 합니다. 입력값: {v}")
        return v


/tmp/ipython-input-1087220116.py:16: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('wake_up_time', 'bed_time')


In [ ]:
from datetime import datetime, time
from zoneinfo import ZoneInfo

def _parse_hhmm(s: str) -> time:
    h, m = map(int, s.strip().split(":"))
    return time(h, m)

def generate_mission_prompt(user_profile: UserProfile):
    # 1) KST 현재 시각
    now = datetime.now(ZoneInfo("Asia/Seoul"))
    current_time = now.strftime("%H:%M")
    current_hour = now.hour

    # 2) 기상/취침 파싱
    wake = _parse_hhmm(user_profile.wake_up_time)
    bed  = _parse_hhmm(user_profile.bed_time)

    # 2-1) **취침 이후면 미션 생성 안 함**
    if now.time() >= bed:
        return None  # 호출부에서 None이면 전송/표시 스킵

    # 3) 시간대 라벨(야간 제외)
    if wake.hour <= current_hour < 12:
        time_period = "오전"
    elif 12 <= current_hour < 18:
        time_period = "오후"
    else:
        time_period = "저녁"

    prompt = f"""
    당신은 노인 복지 전문가입니다. 아래 정보를 반영하여 {time_period} 시간대에 적합한 맞춤 미션 1개를 따뜻하게 제안하세요.

    [사용자 정보]
    - 거동 상태: {'거동 불편' if user_profile.mobility_issue else '거동 가능'}
    - 거주 형태: {'독거' if user_profile.living_arrangement == 'alone' else '가족 동거'}
    - 기상 시간: {user_profile.wake_up_time}
    - 취침 시간: {user_profile.bed_time}
    - 현재 시각(KST): {current_time}
    - 현재 시간대: {time_period}

    [미션 생성 원칙]
    - 안전: 넘어짐/과로 위험 활동 금지. 건강진단 등 의학적 조언 금지.
    - 실행성: 집/근처에서 바로 할 수 있는 활동, 준비물 0~1개.
    - 구체성: 무엇을(명사) 어떻게(동사) 할지, 장소/방법을 명확히.
    - 시간: 10~20분 내 완료 가능(필요시 더 짧은 대안 제시).
    - 대안: 거동 불편 시 앉아서/벽 짚고/짧게 가능한 버전 함께 제시.
    - 사회성: 독거면 노인정 방문/이웃 인사 등 사회 연결 1가지 포함 권장.

    [시간대 고려사항]
    - 오전: 햇볕/환기/가벼운 정리·기지개 등으로 시작
    - 오후: 취미·정리·짧은 외출·사회 활동
    - 저녁: 스트레칭·호흡·감사일기·내일 준비

    [주의 사항]
    - 특수문자·이모지·번호·불릿 사용 금지. 과도한 명령/금지 표현 지양.
    - 날씨/교통 등 외부 조건은 안전을 우선해 실내 대안 함께 제공.
    - 비용/예약이 필요한 활동은 피하고, 무료/무예약 대안 우선.

    [출력 형식 - 매우 중요]
    - 1~2문장으로 전체 60자 이내의 자연스러운 구어체 한국어.
    - 예) 창가에 앉아 가벼운 목, 어깨 스트레칭을 해보세요.
    """
    return prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch

mission_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
mission_prompt = PromptTemplate.from_template("{prompt}")
parser = StrOutputParser()

#UserProfile → (Lambda: prompt/None) → (Branch: None면 끝 / str이면 템플릿→LLM→문자열)
mission_chain = (
    RunnableLambda(lambda profile: generate_mission_prompt(profile))
    | RunnableBranch(
        # 1) None이면 그대로 None 반환 (취침 이후)
        (lambda p: p is None, RunnableLambda(lambda _: None)),
        # 2) 문자열 프롬프트면 LLM 통과
        (lambda p: isinstance(p, str),
            RunnableLambda(lambda p: {"prompt": p})
            | mission_prompt
            | mission_llm
            | parser
        ),
        # 3) 기본(default): 혹시 위 조건에 안 걸리면 None
        RunnableLambda(lambda _: None)
    )
)


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# ===== 1. 스키마 정의 =====
class QuizOutput(BaseModel):
    question: str = Field(description="사용자에게 보여줄 문제")
    answer: str = Field(description="정답")

class GradingOutput(BaseModel):
    correct: bool
    feedback: str

# ===== 2. 파서 생성 =====
quiz_parser = PydanticOutputParser(pydantic_object=QuizOutput)
grading_parser = PydanticOutputParser(pydantic_object=GradingOutput)

# ===== 3. 프롬프트 =====
quiz_prompt_text = '''
[역할]
당신은 고령자 친화형 ‘인지 자극 문제’ 출제자입니다.

[출제 규칙 - 매우 중요]
- 문제는 딱 1개만 만드세요.
- 대상: 고령자. 쉬운 어휘, 존댓말. TTS로 읽힙니다.
- 길이: 문제문은 최대 60자, 줄바꿈·번호·불릿·괄호·특수기호·이모지 금지.
- 의료·법률·위험 활동·개인정보 요구 금지.
- 계산은 초등 1~2학년 수준(한 자리 덧셈/뺄셈 정도).
- 난이도: 기본 very_easy, 필요 시 easy.

[유형 가이드]
- memory: 숫자 거꾸로 말하기, 단어 3개 기억 후 말하기
- attention: 글자 세기(‘가’가 몇 번?), 홀수 찾기(보기로 제시)
- problem_solving: 규칙찾기 또는 순서 중 '가장 먼저 할 일' 고르기

[유형 선택 규칙]
- 아래 3가지 중 **한 가지를 무작위로** 선택해 문제를 1개만 출제하세요.
  memory / attention / problem_solving 중 하나만 사용.

[나쁜 예시]
- “세 자리 곱셈을 풀어보세요”
- “병원에 가서 검사를 받아보세요”
- “①, ②, ( ), #, * 사용”

[좋은 예시]
- question: 숫자 3개를 읽고 거꾸로 말해보세요. 4 7 2
  answer: 2 7 4
- question: 다음 중 과일이 아닌 것은 무엇일까요? 사과 배 우산 바나나
  answer: 우산
- question: 아래 글에서 ‘가’는 몇 번 나오나요? 가나가다라가
  answer: 3
- question: 1, 3, 5 다음 숫자는 무엇일까요?
  answer: 7

[출력 형식]
{format_instructions}
'''

grading_prompt_text = '''
[역할]
당신은 고령자의 음성 답변을 채점하고 따뜻한 피드백을 제공하는 평가자입니다.

[채점 규칙]
- STT로 변환된 답변이므로 발음 변이를 고려하세요
- 의미가 같으면 정답 처리
- 숫자는 다양한 표현 인정 (예: "둘" = "2" = "이")
- 조사나 어미 차이는 무시 (예: "우산" = "우산이요" = "우산입니다")
- 고령자 특성상 추가 설명이 있어도 핵심 답이 맞으면 정답

[입력]
문제: {question}
정답: {correct_answer}
사용자 답변: {user_answer}

[평가 기준]
1. 숫자 답변: 값이 정확히 일치하는가?
2. 단어 답변: 핵심 단어가 포함되어 있는가?
3. 순서 답변: 순서가 정확한가?

[피드백 작성 규칙]
- 고령자 대상이므로 존댓말 사용
- 정답일 때: 칭찬과 격려 (20-30자)
- 오답일 때: 격려하며 정답 알려주기 (30-40자)
- 따뜻하고 긍정적인 어조 유지
- TTS로 읽히므로 자연스러운 문장
- 출력은 correct(Boolean)과 feedback 두 필드만 작성하며, 정답 텍스트는 다시 출력하지 마세요.

[출력 형식]
{format_instructions}
'''

# ===== 4. 체인 구성 =====
quiz_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
grading_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

quiz_prompt = PromptTemplate(
    template=quiz_prompt_text,
    input_variables=[],
    partial_variables={"format_instructions": quiz_parser.get_format_instructions()}
)

grading_prompt = PromptTemplate(
    template=grading_prompt_text,
    input_variables=["question", "correct_answer", "user_answer"],
    partial_variables={"format_instructions": grading_parser.get_format_instructions()}
)

quiz_chain = quiz_prompt | quiz_llm | quiz_parser
grading_chain = grading_prompt | grading_llm | grading_parser

# ===== 5. 서비스 클래스 (심플!) =====
class CognitiveQuizService:
    def __init__(self):
        self.current_quiz = None
        self.quiz_chain = quiz_chain
        self.grading_chain = grading_chain

    def generate_quiz(self):
        """문제 생성"""
        output = self.quiz_chain.invoke({})

        self.current_quiz = {
            'question': output.question,
            'answer': output.answer
        }

        return output.question

    def grade_answer(self, user_answer: str):
        """STT 답변 채점"""
        if not self.current_quiz:
            return {'error': '진행 중인 문제가 없습니다'}

        result = self.grading_chain.invoke({
            'question': self.current_quiz['question'],
            'correct_answer': self.current_quiz['answer'],
            'user_answer': user_answer  # 그대로 전달
        })

        return {
            "is_correct": result.correct,           # ← bool 그대로
            "feedback": result.feedback,
            "user_answer": user_answer,
            "correct_answer": self.current_quiz["answer"]
        }


```서비스 흐름 요약

[generate_quiz()]
├─→ quiz_chain 실행: LLM이 문제와 정답(구조화) 생성
├─→ 파싱/검증: PydanticOutputParser가 이미 구조화(QuizOutput)로 보장
├─→ 상태 저장: self.current_quiz = {question, answer}
└─→ 문제만 반환: 화면/TTS로 노출

사용자 답변 입력

↓
[grade_answer()]
├─→ 상태 확인: 진행 중인 문제 존재 여부 체크
├─→ grading_chain 실행: 저장된 question/correct_answer와 user_answer로 채점
├─→ 파싱/검증: PydanticOutputParser가 구조화(GradingOutput) 보장
├─→ 데이터 조합: is_correct/feedback + user_answer + correct_answer 하나로 합치기
└─→ 통합 결과 반환: 결과 UI/TTS 처리

In [ ]:
# from langchain.output_parsers import PydanticOutputParser
# from pydantic import BaseModel, Field
# from langchain_openai import ChatOpenAI
# from langchain.prompts import PromptTemplate

# # ===== 1. 스키마 정의 =====
# class QuizOutput(BaseModel):
#     question: str = Field(description="사용자에게 보여줄 문제")
#     answer: str = Field(description="정답")

# class GradingOutput(BaseModel):
#     result: str = Field(description="correct 또는 incorrect")
#     feedback: str = Field(description="피드백 메시지")

# # ===== 2. 파서 생성 =====
# quiz_parser = PydanticOutputParser(pydantic_object=QuizOutput)
# grading_parser = PydanticOutputParser(pydantic_object=GradingOutput)

# # ===== 3. 프롬프트 =====
# quiz_prompt_text = '''
# [역할]
# 당신은 고령자 친화형 ‘인지 자극 문제’ 출제자입니다.

# [출제 규칙 - 매우 중요]
# - 문제는 딱 1개만 만드세요.
# - 대상: 고령자. 쉬운 어휘, 존댓말. TTS로 읽힙니다.
# - 길이: 문제문은 최대 60자, 줄바꿈·번호·불릿·괄호·특수기호·이모지 금지.
# - 의료·법률·위험 활동·개인정보 요구 금지.
# - 계산은 초등 1~2학년 수준(한 자리 덧셈/뺄셈 정도).
# - 난이도: 기본 very_easy, 필요 시 easy.

# [유형 가이드]
# - memory: 숫자 거꾸로 말하기, 단어 3개 기억 후 말하기
# - attention: 글자 세기(‘가’가 몇 번?), 홀수 찾기(보기로 제시)
# - problem_solving: 규칙찾기 또는 순서 중 '가장 먼저 할 일' 고르기

# [유형 선택 규칙]
# - 아래 3가지 중 **한 가지를 무작위로** 선택해 문제를 1개만 출제하세요.
#   memory / attention / problem_solving 중 하나만 사용.

# [나쁜 예시]
# - “세 자리 곱셈을 풀어보세요”
# - “병원에 가서 검사를 받아보세요”
# - “①, ②, ( ), #, * 사용”

# [좋은 예시]
# - question: 숫자 3개를 읽고 거꾸로 말해보세요. 4 7 2
#   answer: 2 7 4
# - question: 다음 중 과일이 아닌 것은 무엇일까요? 사과 배 우산 바나나
#   answer: 우산
# - question: 아래 글에서 ‘가’는 몇 번 나오나요? 가나가다라가
#   answer: 3
# - question: 1, 3, 5 다음 숫자는 무엇일까요?
#   answer: 7

# [출력 형식]
# {format_instructions}
# '''

# grading_prompt_text = '''
# [역할]
# 당신은 고령자의 음성 답변을 채점하고 따뜻한 피드백을 제공하는 평가자입니다.

# [채점 규칙]
# - STT로 변환된 답변이므로 발음 변이를 고려하세요
# - 의미가 같으면 정답 처리
# - 숫자는 다양한 표현 인정 (예: "둘" = "2" = "이")
# - 조사나 어미 차이는 무시 (예: "우산" = "우산이요" = "우산입니다")
# - 고령자 특성상 추가 설명이 있어도 핵심 답이 맞으면 정답

# [입력]
# 문제: {question}
# 정답: {correct_answer}
# 사용자 답변: {user_answer}

# [평가 기준]
# 1. 숫자 답변: 값이 정확히 일치하는가?
# 2. 단어 답변: 핵심 단어가 포함되어 있는가?
# 3. 순서 답변: 순서가 정확한가?

# [피드백 작성 규칙]
# - 고령자 대상이므로 존댓말 사용
# - 정답일 때: 칭찬과 격려 (20-30자)
# - 오답일 때: 격려하며 정답 알려주기 (30-40자)
# - 따뜻하고 긍정적인 어조 유지
# - TTS로 읽히므로 자연스러운 문장

# [출력 형식]
# {format_instructions}
# '''

# # ===== 4. 체인 구성 =====
# quiz_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# grading_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# quiz_prompt = PromptTemplate(
#     template=quiz_prompt_text,
#     input_variables=[],
#     partial_variables={"format_instructions": quiz_parser.get_format_instructions()}
# )

# grading_prompt = PromptTemplate(
#     template=grading_prompt_text,
#     input_variables=["question", "correct_answer", "user_answer"],
#     partial_variables={"format_instructions": grading_parser.get_format_instructions()}
# )

# quiz_chain = quiz_prompt | quiz_llm | quiz_parser
# grading_chain = grading_prompt | grading_llm | grading_parser

# # ===== 5. 서비스 클래스 (심플!) =====
# class CognitiveQuizService:
#     def __init__(self):
#         self.current_quiz = None
#         self.quiz_chain = quiz_chain
#         self.grading_chain = grading_chain

#     def generate_quiz(self):
#         """문제 생성"""
#         output = self.quiz_chain.invoke({})

#         self.current_quiz = {
#             'question': output.question,
#             'answer': output.answer
#         }

#         return output.question

#     def grade_answer(self, user_answer: str):
#         """STT 답변 채점"""
#         if not self.current_quiz:
#             return {'error': '진행 중인 문제가 없습니다'}

#         # LLM이 알아서 처리 (전처리 없음)
#         result = self.grading_chain.invoke({
#             'question': self.current_quiz['question'],
#             'correct_answer': self.current_quiz['answer'],
#             'user_answer': user_answer  # 그대로 전달
#         })

#         return {
#             'is_correct': result.result.lower() == 'correct',
#             'feedback': result.feedback,
#             'user_answer': user_answer,
#             'correct_answer': self.current_quiz['answer']
#         }